In [1]:
import sys
import pandas as pd
#import ssbio


from Bio.PDB import PDBParser
from Bio.PDB.DSSP import DSSP

import statistics as stat

from sphere_fibo import sphere_fibo

<class 'numpy.ndarray'>


In [2]:
fichier_pdb='../data/1py6.pdb'


##Constantes

#d'après l'article aa hydrophobes
HYDROPHOBE=["FHE", "GLY", "ILE", "LEU," "MET", "VAL", "TRP","TYR"]
SEUIL_ACC=0.3
NB_POINTS_SPHERE=256


In [3]:
def recup_coord_carbones_alpha(fichier_pdb):
    """
        Recupération des coordonnées des carbones alpha à partir du fichier pdb donné en entrée
        et renvoie d'un dataframe
    """
    data_pdb=[]
    file=open(fichier_pdb,"r")
    
    for ligne in file:
        if ligne.startswith("ATOM"):
            if str.strip(ligne[12:15])=="CA" and (str.strip(ligne[16:17])=="" or str.strip(ligne[16:17])=="A"):
                data_pdb.append([ligne[17:20],float(str.strip(ligne[30:37])),float(str.strip(ligne[38:46])),float(str.strip(ligne[46:53]))])
    file.close()

    data=pd.DataFrame(data=data_pdb,columns=["type_aa","x","y","z"])

    return (data)

In [4]:
data=recup_coord_carbones_alpha(fichier_pdb) 
print(data)

data["hydrophobe"]=data.type_aa.isin(HYDROPHOBE)
print(data["type_aa"])
data

    type_aa      x       y      z
0       THR  11.14  -9.833  19.57
1       GLY  11.61 -12.203  16.63
2       ARG   8.00 -12.293  15.35
3       PRO   7.19 -15.077  12.80
4       GLU   6.53 -12.491  10.09
..      ...    ...     ...    ...
222     ARG   9.04   3.011 -24.88
223     ALA   5.67   4.482 -23.88
224     ILE   7.35   6.666 -21.23
225     PHE   9.06   8.810 -23.85
226     GLY   7.83  12.292 -24.70

[227 rows x 4 columns]
0      THR
1      GLY
2      ARG
3      PRO
4      GLU
      ... 
222    ARG
223    ALA
224    ILE
225    PHE
226    GLY
Name: type_aa, Length: 227, dtype: object


,type_aa,x,y,z,hydrophobe
0,THR,11.14,-9.833,19.57,False
1,GLY,11.61,-12.203,16.63,True
2,ARG,8.00,-12.293,15.35,False
3,PRO,7.19,-15.077,12.80,False
4,GLU,6.53,-12.491,10.09,False
...,...,...,...,...,...
222,ARG,9.04,3.011,-24.88,False
223,ALA,5.67,4.482,-23.88,False
224,ILE,7.35,6.666,-21.23,True
225,PHE,9.06,8.810,-23.85,False


In [19]:
def recup_acc_solvant(fichier_pdb):
    """
    entrée le fichier pdb
    sortie une liste de l'accesibilité relative (en 0 et 1) des résidus
    utilise Biopython et le programme dssp

    """
    print("a")
    acces_solvant=[]
    print("b")
    p = PDBParser()
    structure = p.get_structure("structure",fichier_pdb)
    print(structure)
    model = structure[0]
    print(model)
    dssp = DSSP(model,fichier_pdb)

    for i in range(len(list(dssp))):
        acces_solvant.append((list(dssp)[i][3]))
    return acces_solvant

In [21]:
acces_solvant=recup_acc_solvant(fichier_pdb)

data["acces_solvant"]=acces_solvant 

print(data)

a
b
<Structure id=structure>
<Model id=0>


C:\Users\audod\Anaconda3\lib\site-packages\Bio\PDB\PDBParser.py:345: PDBConstructionWarning: PDBConstructionException: Invalid or missing occupancy at line 1779.
Exception ignored.
Some atoms or residues may be missing in the data structure.
  PDBConstructionWarning,
C:\Users\audod\Anaconda3\lib\site-packages\Bio\PDB\PDBParser.py:345: PDBConstructionWarning: PDBConstructionException: Invalid or missing B factor at line 1779.
Exception ignored.
Some atoms or residues may be missing in the data structure.
  PDBConstructionWarning,
C:\Users\audod\Anaconda3\lib\site-packages\Bio\PDB\Atom.py:205: PDBConstructionWarning: Used element 'N' for Atom (name=N) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
C:\Users\audod\Anaconda3\lib\site-packages\Bio\PDB\PDBParser.py:345: PDBConstructionWarning: PDBConstructionException: Invalid or missing occupancy at line 1780.
Exception ignored.
Some atoms or residues may be missing in the data structure.
  PDBConstructionWarning,
C:\User

FileNotFoundError: [WinError 2] Le fichier spécifié est introuvable

In [26]:
p = PDBParser()
structure = p.get_structure("structure",fichier_pdb)

C:\Users\audod\Anaconda3\lib\site-packages\Bio\PDB\PDBParser.py:345: PDBConstructionWarning: PDBConstructionException: Invalid or missing occupancy at line 1779.
Exception ignored.
Some atoms or residues may be missing in the data structure.
  PDBConstructionWarning,
C:\Users\audod\Anaconda3\lib\site-packages\Bio\PDB\PDBParser.py:345: PDBConstructionWarning: PDBConstructionException: Invalid or missing B factor at line 1779.
Exception ignored.
Some atoms or residues may be missing in the data structure.
  PDBConstructionWarning,
C:\Users\audod\Anaconda3\lib\site-packages\Bio\PDB\Atom.py:205: PDBConstructionWarning: Used element 'N' for Atom (name=N) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
C:\Users\audod\Anaconda3\lib\site-packages\Bio\PDB\PDBParser.py:345: PDBConstructionWarning: PDBConstructionException: Invalid or missing occupancy at line 1780.
Exception ignored.
Some atoms or residues may be missing in the data structure.
  PDBConstructionWarning,
C:\User

In [27]:
DSSP(structure[0],fichier_pdb)

FileNotFoundError: [WinError 2] Le fichier spécifié est introuvable

In [ ]:
#on garde que les résidus avec plus d'un certain seuil d'acessibilité au solvant pour la sphère
data=data.loc[data['acces_solvant'] > SEUIL_ACC]
print(data)

In [ ]:
def recup_centre_sphere(data_frame):
    return([stat.mean(data.x),stat.mean(data.y),stat.mean(data.z)])

In [ ]:
centre_sphere=recup_centre_sphere(data)
print(centre_sphere)

In [28]:
coord_sphere=sphere_fibo(NB_POINTS_SPHERE)
coord_sphere+=centre_sphere ## on décale la sphère sur son centre 

print(coord_sphere)

NameError: name 'centre_sphere' is not defined